# Building AI based Autonomous Traders using mt5se

### For an introduction to mt5se, see: https://paulo-al-castro.github.io/mt5se/


In [ ]:
# installing Metatrader5 using sys
import sys
# python MetaTrader5
#!{sys.executable} -m pip install MetaTrader5
#mt5b3 
!{sys.executable} -m pip install mt5se --upgrade

In [2]:
import mt5se as se
import pandas as pd
connected=se.connect()
if connected:
    print('Ok!! It is connected to se exchange!!')
else:
    print('Something went wrong! It is NOT connected to se!!')



Ok!! It is connected to se exchange!!


## Let's create a Simple Trader based on AI (decision tree)

In [3]:
## Defines the Simple AI Trader
from sklearn import tree
from sklearn.preprocessing import KBinsDiscretizer

class SimpleAITrader(se.Trader):

    def setup(self,dbars):
        assets=list(dbars.keys())
        if len(assets)!=1:
            print('Error, this trader is supposed to deal with just one asset')
            return None
        bars=dbars[assets[0]]
        # remove irrelevant info
        if 'time' in bars:
            del bars['time']
        timeFrame=10 # it takes into account the last 10 bars
        horizon=1 # it project the closing price for next bar
        target='close' # name of the target column
        ds=se.ai_utils.bars2Dataset(bars,target,timeFrame,horizon)
        
        X=se.ai_utils.fromDs2NpArrayAllBut(ds,['target'])
        discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform') 
        # creates the discrete target
        ds['target']=se.ai_utils.discTarget(discretizer,ds['target'])
        Y=se.ai_utils.fromDs2NpArray(ds,['target'])

        clf = tree.DecisionTreeClassifier()

        clf = clf.fit(X, Y)
        self.clf=clf

    def trade(self,dbars):
            assets=dbars.keys()
            orders=[]
            timeFrame=10 # it takes into account the last 10 bars
            horizon=1 # it project the closing price for next bar
            target='close' # name of the target column
            money=se.get_balance()/len(assets) # shares the balance equally among the assets
            for asset in assets:
                bars=dbars[asset]
                curr_shares=se.get_shares(asset)
                price=se.get_last(bars)
                free_shares=se.get_affor_shares(asset,price,money)
                # get new information (bars), transform it in X
                bars=dbars[asset]
                #remove irrelevant info
                if 'time' in bars:
                    del bars['time']
                # convert from bars to dataset
                ds=se.ai_utils.bars2Dataset(bars,target,timeFrame,horizon)
                # Get X fields
                X=se.ai_utils.fromDs2NpArrayAllBut(ds,['target'])

                # predict the result, using the latest info
                p=self.clf.predict([X[-1]])
                if p==2:
                    #buy it
                    order=se.buyOrder(asset,free_shares)
                elif p==0:
                    #sell it
                    order=se.sellOrder(asset,curr_shares)
                else:
                    order=None
                if order!=None:
                    orders.append(order)
            return orders    

# creates instance of the Simple AI Trading
trader=SimpleAITrader()  # also available in se.sampleTraders.SimpleAITrader()
print(trader)


## Setup a backtest!

In [6]:
# sets Backtest options 
prestart=se.date(2018,12,10)
start=se.date(2019,1,10)
end=se.date(2019,2,27)
capital=100000
results_file='data_equity_file.csv'
verbose=False 
assets=['GOOG']
# Use True if you want debug information for your Trader 
#sets the backtest setup
period=se.DAILY 
 # it may be se.INTRADAY (one minute interval)
bts=se.backtest.set(assets,prestart,start,end,period,capital,results_file,verbose)
if se.backtest.checkBTS(bts): # check if the backtest setup is ok!
    print('Backtest Setup is Ok!')
else:
    print('Backtest Setup is NOT Ok!')




Backtest Setup is Ok!


## Run and evaluate the backtest

In [7]:
# Running the backtest
df= se.backtest.run(trader,bts)   
# run calls the Trader. setup and trade (one for bar)

# evaluates the backtest results
se.backtest.evaluate(df)

Esta em backtest. bts= {'verbose': False, 'prestart': datetime.datetime(2018, 12, 10, 0, 0), 'start': datetime.datetime(2019, 1, 10, 0, 0), 'end': datetime.datetime(2019, 2, 27, 0, 0), 'type': 1, 'file': 'data_equity_file.csv', 'assets': ['GOOG'], 'capital': 100000.0, 'shares_GOOG': 0.0, 'curr': 0}
Esta em backtest. bts= {'verbose': False, 'prestart': datetime.datetime(2018, 12, 10, 0, 0), 'start': datetime.datetime(2019, 1, 10, 0, 0), 'end': datetime.datetime(2019, 2, 27, 0, 0), 'type': 1, 'file': 'data_equity_file.csv', 'assets': ['GOOG'], 'capital': 1732.479999999996, 'shares_GOOG': 93.0, 'curr': 1}
Esta em backtest. bts= {'verbose': False, 'prestart': datetime.datetime(2018, 12, 10, 0, 0), 'start': datetime.datetime(2019, 1, 10, 0, 0), 'end': datetime.datetime(2019, 2, 27, 0, 0), 'type': 1, 'file': 'data_equity_file.csv', 'assets': ['GOOG'], 'capital': 1732.479999999996, 'shares_GOOG': 93.0, 'curr': 2}
Esta em backtest. bts= {'verbose': False, 'prestart': datetime.datetime(2018, 12

In [9]:
# One year backtest

# sets Backtest options 
prestart=se.date(2018,12,10)
start=se.date(2019,1,10)
end=se.date(2019,12,31)
capital=100000
results_file='data_equity_file.csv'
verbose=False 
assets=['AAPL']
# Use True if you want debug information for your Trader 
#sets the backtest setup
period=se.DAILY 
 # it may be se.INTRADAY (one minute interval)
bts=se.backtest.set(assets,prestart,start,end,period,capital,results_file,verbose)
if se.backtest.checkBTS(bts): # check if the backtest setup is ok!
    print('Backtest Setup is Ok!')
else:
    print('Backtest Setup is NOT Ok!')
# Running the backtest
df= se.backtest.run(trader,bts)   
# run calls the Trader. setup and trade (one for bar)




 31, 0, 0), 'type': 1, 'file': 'data_equity_file.csv', 'assets': ['AAPL'], 'capital': 38.76999999995405, 'shares_AAPL': 556.0, 'curr': 183}
Esta em backtest. bts= {'verbose': False, 'prestart': datetime.datetime(2018, 12, 10, 0, 0), 'start': datetime.datetime(2019, 1, 10, 0, 0), 'end': datetime.datetime(2019, 12, 31, 0, 0), 'type': 1, 'file': 'data_equity_file.csv', 'assets': ['AAPL'], 'capital': 38.76999999995405, 'shares_AAPL': 556.0, 'curr': 184}
Esta em backtest. bts= {'verbose': False, 'prestart': datetime.datetime(2018, 12, 10, 0, 0), 'start': datetime.datetime(2019, 1, 10, 0, 0), 'end': datetime.datetime(2019, 12, 31, 0, 0), 'type': 1, 'file': 'data_equity_file.csv', 'assets': ['AAPL'], 'capital': 38.76999999995405, 'shares_AAPL': 556.0, 'curr': 185}
Esta em backtest. bts= {'verbose': False, 'prestart': datetime.datetime(2018, 12, 10, 0, 0), 'start': datetime.datetime(2019, 1, 10, 0, 0), 'end': datetime.datetime(2019, 12, 31, 0, 0), 'type': 1, 'file': 'data_equity_file.csv', 'as

## Evaluating Backtesting results

The method backtest.run creates a data file with the name given in the backtest setup (bts) 

This will give you a report about the trader performance

We need ot note that it is hard to perform meaningful evaluations using backtest. There are many pitfalls to avoid and it may be easier to get trading robots with great performance in backtest, but that perform really badly in real operations. 

More about that in mt5b3 backtest evaluation chapter.

For a deeper discussion, we suggest:
Is it a great Autonomous Trading Strategy or you are just fooling yourself Bernardini,M. and Castro, P.A.L

In order to analyze the trader's backtest, you may use :

se.backtest.evaluateFile(fileName)  
#fileName is the name of file generated by the backtest

or

se.bactest.evaluate(df)  # df is the dataframe returned by b3.backtest.run


In [10]:
# evaluates the backtest results
se.backtest.evaluate(df)



 -----------------------   Backtest Report  ------------------------------- 

Total Return (%)=63.40 in 246 bars 
Average Bar Return (%)=0.21  
Std Deviation of returns (%) =1.3602

 ----------------------        End of Report     -------------------------------- 



# Random Forest Trader

In [11]:
## Defines the Simple AI Trader
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import KBinsDiscretizer

class RandomForestTrader(se.Trader):

    def setup(self,dbars):
        assets=list(dbars.keys())
        if len(assets)!=1:
            print('Error, this trader is supposed to deal with just one asset')
            return None
        bars=dbars[assets[0]]
        # remove irrelevant info
        if 'time' in bars:
            del bars['time']
        timeFrame=10 # it takes into account the last 10 bars
        horizon=1 # it project the closing price for next bar
        target='close' # name of the target column
        ds=se.ai_utils.bars2Dataset(bars,target,timeFrame,horizon)
        
        X=se.ai_utils.fromDs2NpArrayAllBut(ds,['target'])
        discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform') 
        # creates the discrete target
        ds['target']=se.ai_utils.discTarget(discretizer,ds['target'])
        Y=se.ai_utils.fromDs2NpArray(ds,['target'])

        #clf = tree.DecisionTreeClassifier()
        clf = RandomForestClassifier(n_estimators=10)
        clf = clf.fit(X, Y)
        self.clf=clf

    def trade(self,dbars):
            assets=dbars.keys()
            orders=[]
            timeFrame=10 # it takes into account the last 10 bars
            horizon=1 # it project the closing price for next bar
            target='close' # name of the target column
            money=se.get_balance()/len(assets) # shares the balance equally among the assets
            for asset in assets:
                bars=dbars[asset]
                curr_shares=se.get_shares(asset)
                price=se.get_last(bars)
                free_shares=se.get_affor_shares(asset,price,money)
                # get new information (bars), transform it in X
                bars=dbars[asset]
                #remove irrelevant info
                if 'time' in bars:
                    del bars['time']
                # convert from bars to dataset
                ds=se.ai_utils.bars2Dataset(bars,target,timeFrame,horizon)
                # Get X fields
                X=se.ai_utils.fromDs2NpArrayAllBut(ds,['target'])

                # predict the result, using the latest info
                p=self.clf.predict([X[-1]])
                if p==2:
                    #buy it
                    order=se.buyOrder(asset,free_shares)
                elif p==0:
                    #sell it
                    order=se.sellOrder(asset,curr_shares)
                else:
                    order=None
                if order!=None:
                    orders.append(order)
            return orders    

# creates instance of the Simple AI Trading
trader=RandomForestTrader()  # also available in se.sampleTraders.SimpleAITrader()
print(trader)

## Setup, run and evaluate backtest

In [12]:


# sets Backtest options 
prestart=se.date(2018,12,10)
start=se.date(2019,1,10)
end=se.date(2019,12,31)
capital=100000
results_file='data_equity_file.csv'
verbose=False 
assets=['GOOG']
# Use True if you want debug information for your Trader 
#sets the backtest setup
period=se.DAILY 
 # it may be se.INTRADAY (one minute interval)
bts=se.backtest.set(assets,prestart,start,end,period,capital,results_file,verbose)

# Running the backtest
df= se.backtest.run(trader,bts)   
# run calls the Trader. setup and trade (one for bar)

# evaluates the backtest results
se.backtest.evaluate(df)




art': datetime.datetime(2019, 1, 10, 0, 0), 'end': datetime.datetime(2019, 12, 31, 0, 0), 'type': 1, 'file': 'data_equity_file.csv', 'assets': ['GOOG'], 'capital': -1773.0799999999872, 'shares_GOOG': 97.0, 'curr': 184}
Esta em backtest. bts= {'verbose': False, 'prestart': datetime.datetime(2018, 12, 10, 0, 0), 'start': datetime.datetime(2019, 1, 10, 0, 0), 'end': datetime.datetime(2019, 12, 31, 0, 0), 'type': 1, 'file': 'data_equity_file.csv', 'assets': ['GOOG'], 'capital': -1773.0799999999872, 'shares_GOOG': 97.0, 'curr': 185}
Esta em backtest. bts= {'verbose': False, 'prestart': datetime.datetime(2018, 12, 10, 0, 0), 'start': datetime.datetime(2019, 1, 10, 0, 0), 'end': datetime.datetime(2019, 12, 31, 0, 0), 'type': 1, 'file': 'data_equity_file.csv', 'assets': ['GOOG'], 'capital': 115502.83000000002, 'shares_GOOG': 0.0, 'curr': 186}
Esta em backtest. bts= {'verbose': False, 'prestart': datetime.datetime(2018, 12, 10, 0, 0), 'start': datetime.datetime(2019, 1, 10, 0, 0), 'end': dateti